<a href="https://colab.research.google.com/github/ankit-rathi/Data-Science-with-Python/blob/master/Stock_Analysis/Stock_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# install libraries
!pip install yfinance
!pip install pandas-ta
#!pip install talib-binary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 6.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas-ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218921 sha256=4123db28783da857f52a4aa558bae14df651f966907ffa6eb10f42d35d9f9ce1
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas-ta


In [2]:
# import libraries 

%matplotlib inline
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')
import pandas_datareader.data as web
import yfinance as yfin
#import talib 
yfin.pdr_override()
import pandas_ta as ta

In [3]:
# set start and end date
import datetime
start_date = (datetime.date.today() + datetime.timedelta(days=-3*365)).strftime('%Y-%m-%d')
end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')

In [4]:
# fetch nifty200 scrips

import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

mypf = pd.read_csv('/content/drive/My Drive/data/stocks/Nifty100Scrips.csv')

mypf = mypf[mypf['InPortfolio'] == 1]

stock_n100 = mypf['Symbol'].unique()

exclude = ['CADILAHC','MMTC']

stock_n100 = mypf[~mypf['Symbol'].isin(exclude) ]['Symbol'].unique()

stock_ids = stock_n100

#mypf = mypf[mypf.Forecast.notnull()]
#stock_ids = mypf['Symbol'].unique()

stock_ids.sort()
stock_ids

Mounted at /content/drive


array(['ASIANPAINT', 'BAJFINANCE', 'BATAINDIA', 'DMART', 'EICHERMOT',
       'GILLETTE', 'HDFCBANK', 'INDIGOPNTS', 'INFY', 'IRCTC', 'JUBLFOOD',
       'PAGEIND', 'PIDILITIND', 'RELIANCE', 'TATACONSUM'], dtype=object)

In [5]:
# function to compute stock attributes

def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20 
    mid_window = 50
    long_window = 150
    moving_avg = 'SMA'
    
    # import the closing price data of the stock for the aforementioned period of time in Pandas dataframe
    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-'))) 
    stock_df = yfin.Ticker(stock_symbol).history(period='3y')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    #stock_df = pd.DataFrame(stock_df) # convert Series object to dataframe 
    #stock_df.columns = {'Close Price'} # assign new colun name
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows 

    # column names for long and short moving average columns
    short_window_col = str(short_window) + '_' + moving_avg
    mid_window_col = str(mid_window) + '_' + moving_avg
    long_window_col = str(long_window) + '_' + moving_avg  

    #print(stock_df)

    # Create short exponential moving average column
    stock_df[short_window_col] = round(stock_df['Close'].rolling(window = short_window, min_periods = 1).mean(),0)

    stock_df[mid_window_col] = round(stock_df['Close'].rolling(window = mid_window, min_periods = 1).mean(),0)     

    #stock_df['RSI'] = round(talib.RSI(stock_df['Close Price'], timeperiod=14),0)

    # Create a long exponential moving average column
    stock_df[long_window_col] = round(stock_df['Close'].rolling(window = long_window, min_periods = 1).mean(),0)
    stock_df['Stock'] = stock_symbol
    
    stock_df['Close'] = round(stock_df['Close'],0)     
    stock_df['Max'] = round(max(stock_df['Close']),0)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'
                   ], axis=1, inplace=True)   
    
    return stock_df

df_prec_dev = pd.DataFrame()

#tmp = stock_prec_dev('ULTRACEMCO.NS')
#df_perform = df_perform.append(tmp)
for stock_id in stock_ids:
    #print(stock_id)
    #current_pe = get_current_pe(stock_id)
    stock_id = stock_id.upper() + '.NS'    
    tmp = stock_prec_dev(stock_id)
    tmp = tmp.reset_index()
    tmp = tmp.tail(1)
    tmp['Avg_Cost'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['AvgCost'].values[0]
    tmp['Shares'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Shares'].values[0]   
    #tmp['Curr_PE'] = current_pe
    df_prec_dev = df_prec_dev.append(tmp)

In [6]:
df_prec_dev.reset_index(drop=True, inplace=True)
df_prec_dev.drop(['Date'], axis=1, inplace=True)

In [7]:
# portfolio stocks

tmp_df = df_prec_dev[df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df['Investment'] = tmp_df['Avg_Cost'] * tmp_df['Shares']
tmp_df['Current'] = tmp_df['Close'] * tmp_df['Shares']
tmp_df['Estimated'] = tmp_df['Max'] * tmp_df['Shares']
tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
tmp_df['Estimated P/L%'] = round((tmp_df['Estimated'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
tmp_df.sort_values(by = 'Current P/L%', ascending=False)

qualified stocks: 15


,Close,20_SMA,50_SMA,150_SMA,Stock,Max,Avg_Cost,Shares,Investment,Current,Estimated,Current P/L%,Estimated P/L%
14,764.0,720.0,715.0,753.0,TATACONSUM.NS,875.0,702.36,40.0,28094.40,30560.0,35000.0,8.78,24.58
1,6280.0,5892.0,5976.0,6461.0,BAJFINANCE.NS,7901.0,5928.85,5.0,29644.25,31400.0,39505.0,5.92,33.26
11,40321.0,38575.0,38073.0,43331.0,PAGEIND.NS,53631.0,38201.00,1.0,38201.00,40321.0,53631.0,5.55,40.39
4,3301.0,3122.0,3118.0,3310.0,EICHERMOT.NS,3850.0,3167.25,6.0,19003.50,19806.0,23100.0,4.22,21.56
13,2420.0,2341.0,2335.0,2449.0,RELIANCE.NS,2811.0,2353.60,13.0,30596.80,31460.0,36543.0,2.82,19.43
0,2902.0,2829.0,2820.0,2997.0,ASIANPAINT.NS,3566.0,2827.88,10.0,28278.80,29020.0,35660.0,2.62,26.10
10,446.0,437.0,443.0,511.0,JUBLFOOD.NS,903.0,439.80,68.0,29906.40,30328.0,61404.0,1.41,105.32
5,4413.0,4341.0,4502.0,4851.0,GILLETTE.NS,5944.0,4353.00,7.0,30471.00,30891.0,41608.0,1.38,36.55
6,1688.0,1657.0,1625.0,1585.0,HDFCBANK.NS,1696.0,1665.30,18.0,29975.40,30384.0,30528.0,1.36,1.84
9,618.0,590.0,605.0,660.0,IRCTC.NS,1164.0,609.75,42.0,25609.50,25956.0,48888.0,1.35,90.90


In [8]:
tmp_df.sort_values(by = 'Estimated P/L%', ascending=False)

,Close,20_SMA,50_SMA,150_SMA,Stock,Max,Avg_Cost,Shares,Investment,Current,Estimated,Current P/L%,Estimated P/L%
7,1154.0,1089.0,1061.0,1257.0,INDIGOPNTS.NS,3112.0,1148.85,20.0,22977.00,23080.0,62240.0,0.45,170.88
10,446.0,437.0,443.0,511.0,JUBLFOOD.NS,903.0,439.80,68.0,29906.40,30328.0,61404.0,1.41,105.32
9,618.0,590.0,605.0,660.0,IRCTC.NS,1164.0,609.75,42.0,25609.50,25956.0,48888.0,1.35,90.90
3,3512.0,3473.0,3447.0,3824.0,DMART.NS,5324.0,3476.95,9.0,31292.55,31608.0,47916.0,1.01,53.12
8,1253.0,1324.0,1422.0,1487.0,INFY.NS,1898.0,1240.70,24.0,29776.80,30072.0,45552.0,0.99,52.98
2,1475.0,1432.0,1427.0,1607.0,BATAINDIA.NS,2173.0,1483.40,14.0,20767.60,20650.0,30422.0,-0.57,46.49
11,40321.0,38575.0,38073.0,43331.0,PAGEIND.NS,53631.0,38201.00,1.0,38201.00,40321.0,53631.0,5.55,40.39
5,4413.0,4341.0,4502.0,4851.0,GILLETTE.NS,5944.0,4353.00,7.0,30471.00,30891.0,41608.0,1.38,36.55
1,6280.0,5892.0,5976.0,6461.0,BAJFINANCE.NS,7901.0,5928.85,5.0,29644.25,31400.0,39505.0,5.92,33.26
0,2902.0,2829.0,2820.0,2997.0,ASIANPAINT.NS,3566.0,2827.88,10.0,28278.80,29020.0,35660.0,2.62,26.10


In [9]:
investment = round(sum(tmp_df['Avg_Cost']*tmp_df['Shares']),0)
current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
curr_pnl_amount = current-investment
curr_pnl_percentage = round((current-investment)*100/investment,2)

estimate = round(sum(tmp_df['Max']*tmp_df['Shares']),0)
est_pnl_amount = estimate-investment
est_pnl_percentage = round((estimate-investment)*100/investment,2)

print('Investment: ', investment)
print('Current: ', current)
print('Current Profit/Loss: ', curr_pnl_amount)
print('Current Profit/Loss %: ', curr_pnl_percentage)
print('Estimate: ', estimate)
print('Estimate Profit/Loss: ', est_pnl_amount)
print('Estimate Profit/Loss %: ', est_pnl_percentage)

Investment:  413955.0
Current:  424888.0
Current Profit/Loss:  10933.0
Current Profit/Loss %:  2.64
Estimate:  615213.0
Estimate Profit/Loss:  201258.0
Estimate Profit/Loss %:  48.62


In [10]:
tmp_df['Weightage%'] = round(tmp_df['Investment']*100/investment,2)
tmp_df.sort_values(by = 'Weightage%', ascending=False)

,Close,20_SMA,50_SMA,150_SMA,Stock,Max,Avg_Cost,Shares,Investment,Current,Estimated,Current P/L%,Estimated P/L%,Weightage%
11,40321.0,38575.0,38073.0,43331.0,PAGEIND.NS,53631.0,38201.00,1.0,38201.00,40321.0,53631.0,5.55,40.39,9.23
3,3512.0,3473.0,3447.0,3824.0,DMART.NS,5324.0,3476.95,9.0,31292.55,31608.0,47916.0,1.01,53.12,7.56
13,2420.0,2341.0,2335.0,2449.0,RELIANCE.NS,2811.0,2353.60,13.0,30596.80,31460.0,36543.0,2.82,19.43,7.39
5,4413.0,4341.0,4502.0,4851.0,GILLETTE.NS,5944.0,4353.00,7.0,30471.00,30891.0,41608.0,1.38,36.55,7.36
6,1688.0,1657.0,1625.0,1585.0,HDFCBANK.NS,1696.0,1665.30,18.0,29975.40,30384.0,30528.0,1.36,1.84,7.24
10,446.0,437.0,443.0,511.0,JUBLFOOD.NS,903.0,439.80,68.0,29906.40,30328.0,61404.0,1.41,105.32,7.22
8,1253.0,1324.0,1422.0,1487.0,INFY.NS,1898.0,1240.70,24.0,29776.80,30072.0,45552.0,0.99,52.98,7.19
1,6280.0,5892.0,5976.0,6461.0,BAJFINANCE.NS,7901.0,5928.85,5.0,29644.25,31400.0,39505.0,5.92,33.26,7.16
0,2902.0,2829.0,2820.0,2997.0,ASIANPAINT.NS,3566.0,2827.88,10.0,28278.80,29020.0,35660.0,2.62,26.10,6.83
14,764.0,720.0,715.0,753.0,TATACONSUM.NS,875.0,702.36,40.0,28094.40,30560.0,35000.0,8.78,24.58,6.79


In [11]:
tmp_df.to_csv('/content/drive/My Drive/data/stocks/Stock-Portfolio-Analysis.csv', index=False)

In [12]:
stock_ids = tmp_df['Stock'].values
for stock_id in stock_ids:
    print(stock_id)
    
    df = yfin.Ticker(stock_id).history(period='18mo', interval = "1wk")[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]
    print(round(df['Close'].mean()))
    print(round(df.tail(1)['Close'].values[0]))

ASIANPAINT.NS
3068
2894
BAJFINANCE.NS
6651
6263
BATAINDIA.NS
1747
1472
DMART.NS
4039
3514
EICHERMOT.NS
2956
3298
GILLETTE.NS
5001
4394
HDFCBANK.NS
1499
1687
INDIGOPNTS.NS
1545
1154
INFY.NS
1565
1253
IRCTC.NS
708
618
JUBLFOOD.NS
570
447
PAGEIND.NS
42969
40321
PIDILITIND.NS
2445
2415
RELIANCE.NS
2478
2420
TATACONSUM.NS
757
762
